In [1]:
import collections 
import copy 
import csv 
import os 
import modeling 
import optimization 
import tokenization 
import tensorflow as tf
import six
import math

In [2]:
init_checkpoint = "/home/raghul/Documents/CS7016/mbert-tdl/multi_cased_L-12_H-768_A-12/bert_model.ckpt"

In [3]:
vars = tf.train.list_variables(init_checkpoint)
vars

[('bert/embeddings/LayerNorm/beta', [768]),
 ('bert/embeddings/LayerNorm/gamma', [768]),
 ('bert/embeddings/position_embeddings', [512, 768]),
 ('bert/embeddings/token_type_embeddings', [2, 768]),
 ('bert/embeddings/word_embeddings', [119547, 768]),
 ('bert/encoder/layer_0/attention/output/LayerNorm/beta', [768]),
 ('bert/encoder/layer_0/attention/output/LayerNorm/gamma', [768]),
 ('bert/encoder/layer_0/attention/output/dense/bias', [768]),
 ('bert/encoder/layer_0/attention/output/dense/kernel', [768, 768]),
 ('bert/encoder/layer_0/attention/self/key/bias', [768]),
 ('bert/encoder/layer_0/attention/self/key/kernel', [768, 768]),
 ('bert/encoder/layer_0/attention/self/query/bias', [768]),
 ('bert/encoder/layer_0/attention/self/query/kernel', [768, 768]),
 ('bert/encoder/layer_0/attention/self/value/bias', [768]),
 ('bert/encoder/layer_0/attention/self/value/kernel', [768, 768]),
 ('bert/encoder/layer_0/intermediate/dense/bias', [3072]),
 ('bert/encoder/layer_0/intermediate/dense/kernel'

In [4]:
class InputExample(object):
  """A single training/test example for simple sequence classification."""

  def __init__(self, guid, text_a, text_b=None, label=None):
    """Constructs a InputExample.

    Args:
      guid: Unique id for the example.
      text_a: string. The untokenized text of the first sequence. For single
        sequence tasks, only this sequence must be specified.
      text_b: (Optional) string. The untokenized text of the second sequence.
        Only must be specified for sequence pair tasks.
      label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """
    self.guid = guid
    self.text_a = text_a
    self.text_b = text_b
    self.label = label


class PaddingInputExample(object):
  """Fake example so the num input examples is a multiple of the batch size.

  When running eval/predict on the TPU, we need to pad the number of examples
  to be a multiple of the batch size, because the TPU requires a fixed batch
  size. The alternative is to drop the last batch, which is bad because it means
  the entire output data won't be generated.

  We use this class instead of `None` because treating `None` as padding
  battches could cause silent errors.
  """


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               is_real_example=True):
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.label_id = label_id
    self.is_real_example = is_real_example


class DataProcessor(object):
  """Base class for data converters for sequence classification data sets."""

  def get_train_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the train set."""
    raise NotImplementedError()

  def get_dev_examples(self, data_dir):
    """Gets a collection of `InputExample`s for the dev set."""
    raise NotImplementedError()

  def get_test_examples(self, data_dir):
    """Gets a collection of `InputExample`s for prediction."""
    raise NotImplementedError()

  def get_labels(self):
    """Gets the list of labels for this data set."""
    raise NotImplementedError()

  @classmethod
  def _read_tsv(cls, input_file, quotechar=None):
    """Reads a tab separated value file."""
    with tf.gfile.Open(input_file, "r") as f:
      reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
      lines = []
      for line in reader:
        lines.append(line)
      return lines

In [5]:
class MrpcProcessor(DataProcessor):
  """Processor for the MRPC data set (GLUE version)."""

  def get_train_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

  def get_dev_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

  def get_test_examples(self, data_dir):
    """See base class."""
    return self._create_examples(
        self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

  def get_labels(self):
    """See base class."""
    return ["0", "1"]

  def _create_examples(self, lines, set_type):
    """Creates examples for the training and dev sets."""
    examples = []
    for (i, line) in enumerate(lines):
      if i == 0:
        continue
      guid = "%s-%s" % (set_type, i)
      text_a = tokenization.convert_to_unicode(line[3])
      text_b = tokenization.convert_to_unicode(line[4])
      if set_type == "test":
        label = "0"
      else:
        label = tokenization.convert_to_unicode(line[0])
      examples.append(
          InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    return examples

In [6]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

In [7]:
def convert_single_example(ex_index, example, label_list, max_seq_length,
                           tokenizer):
  """Converts a single `InputExample` into a single `InputFeatures`."""

  if isinstance(example, PaddingInputExample):
    return InputFeatures(
        input_ids=[0] * max_seq_length,
        input_mask=[0] * max_seq_length,
        segment_ids=[0] * max_seq_length,
        label_id=0,
        is_real_example=False)

  label_map = {}
  for (i, label) in enumerate(label_list):
    label_map[label] = i

  tokens_a = tokenizer.tokenize(example.text_a)
  tokens_b = None
  if example.text_b:
    tokens_b = tokenizer.tokenize(example.text_b)

  if tokens_b:
    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
  else:
    # Account for [CLS] and [SEP] with "- 2"
    if len(tokens_a) > max_seq_length - 2:
      tokens_a = tokens_a[0:(max_seq_length - 2)]

  # The convention in BERT is:
  # (a) For sequence pairs:
  #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
  #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
  # (b) For single sequences:
  #  tokens:   [CLS] the dog is hairy . [SEP]
  #  type_ids: 0     0   0   0  0     0 0
  #
  # Where "type_ids" are used to indicate whether this is the first
  # sequence or the second sequence. The embedding vectors for `type=0` and
  # `type=1` were learned during pre-training and are added to the wordpiece
  # embedding vector (and position vector). This is not *strictly* necessary
  # since the [SEP] token unambiguously separates the sequences, but it makes
  # it easier for the model to learn the concept of sequences.
  #
  # For classification tasks, the first vector (corresponding to [CLS]) is
  # used as the "sentence vector". Note that this only makes sense because
  # the entire model is fine-tuned.
  tokens = []
  segment_ids = []
  tokens.append("[CLS]")
  segment_ids.append(0)
  for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
  tokens.append("[SEP]")
  segment_ids.append(0)

  if tokens_b:
    for token in tokens_b:
      tokens.append(token)
      segment_ids.append(1)
    tokens.append("[SEP]")
    segment_ids.append(1)

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1] * len(input_ids)

  # Zero-pad up to the sequence length.
  while len(input_ids) < max_seq_length:
    input_ids.append(0)
    input_mask.append(0)
    segment_ids.append(0)

  assert len(input_ids) == max_seq_length
  assert len(input_mask) == max_seq_length
  assert len(segment_ids) == max_seq_length

  label_id = label_map[example.label]
  if ex_index < 5:
    tf.logging.info("*** Example ***")
    tf.logging.info("guid: %s" % (example.guid))
    tf.logging.info("tokens: %s" % " ".join(
        [tokenization.printable_text(x) for x in tokens]))
    tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
    tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
    tf.logging.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
    tf.logging.info("label: %s (id = %d)" % (example.label, label_id))

  feature = InputFeatures(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids,
      label_id=label_id,
      is_real_example=True)
  return feature

In [8]:
def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer):
  """Convert a set of `InputExample`s to a list of `InputFeatures`."""

  features = []
  for (ex_index, example) in enumerate(examples):
    if ex_index % 10000 == 0:
      tf.logging.info("Writing example %d of %d" % (ex_index, len(examples)))

    feature = convert_single_example(ex_index, example, label_list,
                                     max_seq_length, tokenizer)

    features.append(feature)
  return features

In [9]:
processor = MrpcProcessor()

In [10]:
bert_base_dir = "/home/raghul/Documents/CS7016/mbert-tdl/multi_cased_L-12_H-768_A-12"
glue_dir = "/home/raghul/Documents/CS7016/mbert-tdl/glue_data"

bert_config_file = bert_base_dir + "/bert_config.json"
vocab_file = bert_base_dir + "/vocab.txt"
data_dir = glue_dir + "/MRPC"
init_checkpoint = bert_base_dir + "/bert_model.ckpt"

In [11]:
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

In [12]:
tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=False)

In [13]:
train_examples = processor.get_train_examples(data_dir)

In [14]:
train_features = convert_examples_to_features(train_examples[12:14], ['0', '1'], 128, tokenizer)
train_features

INFO:tensorflow:Writing example 0 of 2
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train-13
INFO:tensorflow:tokens: [CLS] But he added group performance would improve in the second half of the year and beyond . [SEP] De Sole said in the results statement that group performance would improve in the second half of the year and beyond . [SEP]
INFO:tensorflow:input_ids: 101 16976 10261 16288 11795 14432 10894 33992 10106 10105 11132 13877 10108 10105 10924 10111 28569 119 102 10190 67478 12415 10106 10105 17466 33311 10189 11795 14432 10894 33992 10106 10105 11132 13877 10108 10105 10924 10111 28569 119 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [15]:
def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 labels, num_labels, use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  return model

In [16]:
input_ids = tf.convert_to_tensor([feature.input_ids for feature in train_features])
input_mask = tf.convert_to_tensor([feature.input_mask for feature in train_features])
segment_ids = tf.convert_to_tensor([feature.segment_ids for feature in train_features])
label_ids = tf.convert_to_tensor([feature.label_id for feature in train_features])

model = create_model(
        bert_config, False, input_ids, input_mask, segment_ids, label_ids,
        2, False)

In [17]:
def transpose_for_scores(input_tensor, batch_size, num_attention_heads,
                           seq_length, width):
    output_tensor = tf.reshape(
        input_tensor, [batch_size, seq_length, num_attention_heads, width])

    output_tensor = tf.transpose(output_tensor, [0, 2, 1, 3])
    return output_tensor

In [18]:
def get_attention_weights(layer, model, config, batch_size, seq_length, token_type_ids, size_per_head, attention_mask):
    with tf.variable_scope("layer_%d" % layer, reuse = True):
        with tf.variable_scope("attention", reuse = True):
            with tf.variable_scope("self", reuse = True):
                query_layer = tf.layers.dense(
                  model.embedding_output,
                  config.hidden_size,
                  activation=None,
                  name="query")

                # `key_layer` = [B*T, N*H]
                key_layer = tf.layers.dense(
                  model.embedding_output,
                  config.hidden_size,
                  activation=None,
                  name="key")

                # `value_layer` = [B*T, N*H]
                value_layer = tf.layers.dense(
                  model.embedding_output,
                  config.hidden_size,
                  activation=None,
                  name="value")

                # `query_layer` = [B, N, F, H]
                query_layer = transpose_for_scores(query_layer, batch_size,
                             config.num_attention_heads, seq_length,
                             size_per_head)

                # `key_layer` = [B, N, T, H]
                key_layer = transpose_for_scores(key_layer, batch_size, config.num_attention_heads,
                           seq_length, size_per_head)

                # Take the dot product between "query" and "key" to get the raw
                # attention scores.
                # `attention_scores` = [B, N, F, T]
                attention_scores = tf.matmul(query_layer, key_layer, transpose_b=True)
                attention_scores = tf.multiply(attention_scores,
                                               1.0 / math.sqrt(float(size_per_head)))
                if attention_mask is not None:
                  # `attention_mask` = [B, 1, F, T]
                  attention_mask = tf.expand_dims(attention_mask, axis=[1])

                  # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
                  # masked positions, this operation will create a tensor which is 0.0 for
                  # positions we want to attend and -10000.0 for masked positions.
                  adder = (1.0 - tf.cast(attention_mask, tf.float32)) * -10000.0

                  # Since we are adding it to the raw scores before the softmax, this is
                  # effectively the same as removing these entirely.
                  attention_scores += adder
                # Normalize the attention scores to probabilities.
                # `attention_probs` = [B, N, F, T]
                attention_probs = tf.nn.softmax(attention_scores)
                return attention_probs

In [19]:
config = copy.deepcopy(bert_config)
config.hidden_dropout_prob = 0.0
config.attention_probs_dropout_prob = 0.0

input_shape = modeling.get_shape_list(input_ids, expected_rank=2)
batch_size = input_shape[0]
seq_length = input_shape[1]

token_type_ids = segment_ids

size_per_head = int(config.hidden_size/config.num_attention_heads)

attention_probs_all_layers = []

with tf.variable_scope("bert", reuse=True):
  with tf.variable_scope("encoder", reuse=True):
        # This converts a 2D mask of shape [batch_size, seq_length] to a 3D
        # mask of shape [batch_size, seq_length, seq_length] which is used
        # for the attention scores.
        attention_mask = modeling.create_attention_mask_from_input_mask(
            input_ids, input_mask)
        for i in range(12):
            attention_probs = get_attention_weights(i,
                                                    model,
                                                    config,
                                                    batch_size,
                                                    seq_length,
                                                    token_type_ids,
                                                    size_per_head,
                                                    attention_mask)
            attention_probs_all_layers.append(attention_probs)
            
attention_probs_all_layers = tf.stack(attention_probs_all_layers, axis = 1)

In [20]:
tvars = tf.trainable_variables()
print(tvars)

(assignment_map, initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

[<tf.Variable 'bert/embeddings/word_embeddings:0' shape=(119547, 768) dtype=float32_ref>, <tf.Variable 'bert/embeddings/token_type_embeddings:0' shape=(2, 768) dtype=float32_ref>, <tf.Variable 'bert/embeddings/position_embeddings:0' shape=(512, 768) dtype=float32_ref>, <tf.Variable 'bert/embeddings/LayerNorm/beta:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'bert/embeddings/LayerNorm/gamma:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/query/kernel:0' shape=(768, 768) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/query/bias:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/key/kernel:0' shape=(768, 768) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/key/bias:0' shape=(768,) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/value/kernel:0' shape=(768, 768) dtype=float32_ref>, <tf.Variable 'bert/encoder/layer_0/attention/self/value/bias:0' shap

In [21]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    probs = sess.run(attention_probs_all_layers)

In [22]:
print(probs.shape)

(2, 12, 12, 128, 128)


In [26]:
#example number, layer, head, token number
probs[1, 11, 11, 4]

array([0.01225172, 0.01315385, 0.00439894, 0.00382379, 0.21517873,
       0.00495163, 0.00187129, 0.01400201, 0.00470753, 0.01748784,
       0.00187349, 0.00386996, 0.00365059, 0.00429015, 0.00672771,
       0.0029812 , 0.00319304, 0.00128783, 0.00909213, 0.00652297,
       0.00213521, 0.00420338, 0.00137108, 0.00107538, 0.00158061,
       0.00447794, 0.00576828, 0.02096051, 0.00406371, 0.0077876 ,
       0.00584981, 0.00653663, 0.0018854 , 0.00498847, 0.00528718,
       0.00815405, 0.01259863, 0.00416539, 0.00449426, 0.00151985,
       0.00984372, 0.00861141, 0.00418927, 0.00967541, 0.00376012,
       0.00288444, 0.00270182, 0.00803264, 0.00995588, 0.02952315,
       0.01972046, 0.022369  , 0.00357656, 0.00240939, 0.00479007,
       0.00813237, 0.0056687 , 0.00800696, 0.00426274, 0.0043012 ,
       0.36638713, 0.00581693, 0.01116088, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     